In [ ]:
import conversion_tools as ct
def test_convert_units():

    # 1 kg = 1000 g
    assert ct.convert_unit(quant=1, units_in='kg', units_out='g',
                           units = { 'kg': 1, 'g': 1000 }) == 1000
    
    assert ct.convert_bytes(1024, 'B', 'BIT') == 1024 * 8
    assert ct.convert_bytes(1024, 'KB', 'MB') == 1

test_convert_units()

In [ ]:
import conversion_tools as ct
def test_duration_parser():
    assert ct.parse_duration('1 minute, 10 seconds, and 1 hour')[1].total_seconds() == 10 + 60 + 3600 
    assert ct.parse_duration('1 minute, 10 seconds, and 1 hour')[1].days == 0
    assert ct.parse_duration('1 minute, 10 seconds, and 1 hour')[1].seconds == 60 + 10 + 3600
    assert(ct.parse_duration('11 hour and 125 minutes and 12 hours and 1 day')[0].days == 2)
    assert(ct.parse_duration('11 hour and 125 minutes and 12 hours and 1 day')[0].hours == 1)
    assert(ct.parse_duration('11 hour and 125 minutes and 12 hours and 1 day')[0].minutes == 5)
    # 11 hours + 125 minutes + 12 hours + 1 day = 2 days, 1 hour, 5 minutes
    print(ct.parse_duration('11 hour and 125 minutes and 12 hours and 1 day')[0])
    assert(ct.parse_duration('2 weeks, 7 days, and 168 hours')[0].days == 28)


test_duration_parser()

In [ ]:
import json_cache as jc

def test_json_cache():
    duration = 2
    cache = jc.JsonCache("./test_cache.json", duration=duration)

    print("Loading cache with data and initial duration of {} seconds".format(duration))
    cache.save({"test": "data"})
    assert cache.load() == {"test": "data"}

    print(cache.load())
    print("Remaining time: {}".format(cache.get_time_remaining()))

    import time
    time.sleep(cache.get_time_remaining() / 2)
    assert cache.get_time_remaining() <= duration / 2

    print(cache.load())
    print("Remaining time: {}".format(cache.get_time_remaining()))
    time.sleep(duration / 2)
    print("Remaining time: {}".format(cache.get_time_remaining()))
    time.sleep(duration / 2)
    try:
        cache.load()
        assert False
    except RuntimeError as e:
        print("Cache expired as expected")

    assert cache.get_time_remaining() == 0
    print("Test passed")

test_json_cache()

In [ ]:
import ollama_data_utils as odu
import json


def pp(data):
    if isinstance(data, dict):
        data.pop('modelfile', None)
    print(json.dumps(data, indent=4))

def test_ollama_data_utils():
    m = odu.get_models()[1]
    m['modelfile'] = None
    print(json.dumps(m, indent=4))
    #print(m['age'])
    #print(json.dumps(m['weights_file_info'], indent=4))
    
    #print(json.dumps(odu.get_models(), indent=4))

test_ollama_data_utils()

In [21]:
import ollama_data as od
import json
data = od.OllamaData(cache_path='./test/test_cache3.json', cache_time='30 second')

def pp(data):
    if isinstance(data, dict):
        data.pop('modelfile', None)
    print(json.dumps(data, indent=4))


ms = data.get_model('phi')
pp(ms)

{
    "name": "phi:latest",
    "last_modified": "2024-02-15T15:32:38.390715",
    "age": {
        "days": 0,
        "seconds": 0,
        "years": 0,
        "months": 3,
        "weeks": 0,
        "hours": 0,
        "minutes": 0
    },
    "model_params": {
        "stop": "\"System:\""
    },
    "system_message": "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful answers to the user's questions.\n",
    "template": [
        "{{ if .System }}System: {{ .System }}{{ end }}",
        "User: {{ .Prompt }}",
        "Assistant:"
    ],
    "total_weights_size": 1.4924086034297943,
    "total_weights_size_units": "GB",
    "weights": [
        {
            "file_name": "sha256-04778965089b91318ad61d0995b7e44fad4b9a9f4e049d7be90932bf8812e828",
            "file_path": "/usr/share/ollama/.ollama/models/blobs/sha256-04778965089b91318ad61d0995b7e44fad4b9a9f4e049d7be90932bf8812e828",
            "file_size": 1602461536.0,
            "fi

In [ ]:
results = data.search('[*].name')
pp(results)

In [ ]:
pp(data[0])

In [ ]:
data.get_schema()

data.search("[?name == 'llama3:latest'].weights")




In [ ]:
data.search("[?last_modified > '2024-03-16T00:00:00Z'].{name: name, modified: last_modified}")


In [ ]:

data.search("[0:3].{name: name, size: weights_size}")

q = "[?total_weights_size > `7` && total_weights_size_units == 'GB'].{name: name, size: join('', [to_string(total_weights_size), ' ', total_weights_size_units])}"
data.search(q)


In [ ]:

pp(data[0])

In [ ]:
q = "[?last_modified > '2024-01-11T00:00:00Z']"
data.search(q)

In [ ]:
q = "[?age.days > `1`].{name: name, age: age}"
data.search(q, regex="mistral", regex_path="name")
 

In [ ]:
data.search('[*].{name: name}', regex="mistr|phi", regex_path="name")

In [ ]:
print(json.dumps(data.search('[1:20].{name: name, size: total_weights_size}', regex="mistral", regex_path="name"), indent=4))